# Introduction to this Notebook

This Jupyter Notebook encompassess a series of scripts written in Python by Daniel Teixeira dos Santos, a Data Community Innovator at the Data Community of Practice ([link to my forum account](https://rcop.michaeljfox.org/u/danieltds/summary)). These scripts were written using data from PPMI, obtained through LONI. These files are linked to the MJFF Research Community's GitHub repository ([link here](https://github.com/MJFF-ResearchCommunity/Useful-PPMI-Clinical-Codes))

The goal of these scripts is to provide researchers some relevant clinical data that are extracted in a meaningful way from the data that is already available in PPMI. All the necessary input datasets can be obtained [here](https://ida.loni.usc.edu/pages/access/studyData.jsp?project=PPMI) after applying for registration for access to the PPMI data. All outputs from the analyses were removed to comply with privacy and data sharing principles. Some of these scripts were developed with the help of AI tools such as ChatGPT 4o. However, all code was revised and confirmed was working as intended.

This analysis requires two different folders to exist within the main folder. Those are "data" and "priv". The "data" folder is the place where you should store your datasets downloaded from LONI. The priv folder is the one the results will be exported to. These folders will be generated automatically at the beginning of this script, if they don't exist.

# Importing and Setting Paths

In [17]:
import os
import pandas as pd
import numpy as np
import warnings
import sys

#add path to utils folder with shared functions
sys.path.append("../utils")
from helpers import get_latest_file, safe_to_numeric

# Automatically find the "Useful PPMI Clinical Codes" directory
CURRENT_DIR = os.getcwd()
while not CURRENT_DIR.endswith("Useful-PPMI-Clinical-Codes") and os.path.dirname(CURRENT_DIR) != CURRENT_DIR:
    CURRENT_DIR = os.path.dirname(CURRENT_DIR)

BASE_DIR = CURRENT_DIR

# Define paths for "data" and "report" directories
DATA_DIR = os.path.join(BASE_DIR, "data")
PRIV_DIR = os.path.join(BASE_DIR, "priv")

# Ensure both directories exist, create them if not
for directory in [DATA_DIR, PRIV_DIR]:
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created missing folder: {directory}")
    else:
        print(f"Found folder: {directory}")

# Ignore persistent warnings
warnings.simplefilter("ignore", UserWarning)

# Configure Pandas for better data visualization
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = "{:,.3f}".format

# List available files in both directories
print("Files in data directory:", os.listdir(DATA_DIR))
print("Files in priv directory:", os.listdir(PRIV_DIR))


Found folder: G:\Meu Drive\Colab\Useful-PPMI-Clinical-Codes\data
Found folder: G:\Meu Drive\Colab\Useful-PPMI-Clinical-Codes\priv
Files in data directory: ['LEDD_Concomitant_Medication_Log_12Jan2025.csv', 'Concomitant_Medication_Log_09Feb2025.csv', 'Medical_Conditions_Log_09Feb2025.csv', 'MDS-UPDRS_Part_III_09Feb2025.csv', 'Procedure_for_PD_Log_01Aug2025.csv', 'PD_Diagnosis_History_07Oct2024.csv', 'Participant_Status_01Aug2025.csv']
Files in priv directory: ['Medical_Conditions_Charlson.csv', 'Non_PD_Medications_GLP1.csv', 'LEDD_by_timepoints_and_med_types.csv', 'Levodopa_challenge_no_DBS.csv', 'Levodopa_challenge_DBS.csv', 'Hirotaka Notes.txt', 'How to run this.txt', 'Useful Links.txt', 'Clinical Analyses.ipynb', '.ipynb_checkpoints', 'DBS_surgery_cross_longitudinal.csv', 'DBS_surgery_cross_sectional.csv', 'Deceased_patients_cross_sectional.csv', 'Deceased_patients_longitudinal_duration_at_death.csv']


# Longitudinal DBS Status (PPMI)

Deep Brain Stimulation (DBS) is one major PD treatment for patients with tremor motor and/or motor complications that are difficult to manage with medications alone. It is expected that 20% of PD patients could be candidates and undergo this procedure during the disease duration. PPMI colects longitudinal data on DBS surgery from it's participating. Extracting it and correlating to several other variables could provide usefull in predicting undergoing this procedure, or other types of analyses.

**Necessary PPMI datasets:** Procedure_for_PD_Log, PD_Diagnosis_History, Participant_Status and MDS-UPDRS Part III Motor Examination

**Last Update:** November 11, 2025

## Loading and cleaning the original PPMI DBS dataset

Goal of this part: standardize this dataset so that it can be later merged with other datasets such as the MDS III. We also only want to keep patients that have underwent DBS surgery and, for those that didn't, add them to this dataset as not having going through it

In [18]:
DBS_FILE = get_latest_file(prefix="Procedure_for_PD_Log", directory=DATA_DIR)
dbs = pd.read_csv(DBS_FILE)
print('Length of the dataset:', len(dbs))
df.head()

Looking for files in: G:\Meu Drive\Colab\Useful-PPMI-Clinical-Codes\data
Found files: ['G:\\Meu Drive\\Colab\\Useful-PPMI-Clinical-Codes\\data\\Procedure_for_PD_Log_01Aug2025.csv']
Latest file: G:\Meu Drive\Colab\Useful-PPMI-Clinical-Codes\data\Procedure_for_PD_Log_01Aug2025.csv
Length of the dataset: 196


,REC_ID,PATNO,EVENT_ID,PAG_NAME,PDSURGDT,PDSURGTP,PDSRGTPC,PDSURGSD,PDSRGLOC,ORIG_ENTRY,LAST_UPDATE
0,IA98081,3010,ED,SURGPD,08/2016,1.000,NaN,1.000,5.000,12/2021,2022-11-07 00:00:00.0
1,703434601,3010,V14,SURGPD,08/2016,1.000,NaN,1.000,1.000,08/2018,2020-06-30 09:26:01.0
2,760028801,3010,V15,SURGPD,08/2016,1.000,NaN,1.000,1.000,11/2019,2020-06-30 09:26:12.0
3,IA445027,3061,ED,SURGPD,12/2015,1.000,NaN,1.000,5.000,03/2024,2024-03-14 00:00:00.0
4,562809401,3061,V12,SURGPD,12/2016,1.000,NaN,1.000,2.000,04/2016,2020-06-24 12:35:02.0


Mergining

Is there anyone with NaN in EVENT_ID but a not NAN value in PD surgery? Answer: no

In [19]:
df['EVENT_ID'].value_counts(dropna=False)

EVENT_ID
ED     147
V14      8
V11      7
V12      6
V10      5
V13      5
V06      4
V04      4
SC       4
V05      4
V15      4
V08      3
V09      3
BL       2
V07      2
T15      1
T27      1
Name: count, dtype: int64

In [20]:
df['PDSURGTP'].value_counts(dropna=False)

PDSURGTP
1.000    191
3.000     11
2.000      7
5.000      1
Name: count, dtype: int64

There is one NaN here. Let's check

In [21]:
df[df['PDSURGTP'].isna()]

,REC_ID,PATNO,EVENT_ID,PAG_NAME,PDSURGDT,PDSURGTP,PDSRGTPC,PDSURGSD,PDSRGLOC,ORIG_ENTRY,LAST_UPDATE


It is likely this patient underwent DBS, but we can't presume this, therefore, this patient will be removed for our analysis

In [22]:
subset = df[df['EVENT_ID'].isna() & df['PDSURGTP'].notna()]
subset

,REC_ID,PATNO,EVENT_ID,PAG_NAME,PDSURGDT,PDSURGTP,PDSRGTPC,PDSURGSD,PDSRGLOC,ORIG_ENTRY,LAST_UPDATE


Let's replace everyone that doesn't have info with EVENT_ID = 'ED' with 'PDSURGTP' = 0 (those are patients that didn't went DBS, as shown above)

In [23]:
# Remove rows where PDSURGTP is NaN and update df
before = df.shape[0]

df = df[df["PDSURGTP"].notna()].copy()

after = df.shape[0]
print(f"Rows before filtering: {before}")
print(f"Rows after filtering:  {after}")
print(f"Rows removed:          {before - after}")

Rows before filtering: 210
Rows after filtering:  210
Rows removed:          0


Since EVENT_ID timepoint "ED" already provides with with the information we need, let's just confirm if everyone has an ED EVENT_ID, if not, let's just rename

In [24]:
# Step 1: Group all EVENT_IDs per patient
event_ids_per_patient = df.groupby('PATNO')['EVENT_ID'].unique()

# Step 2: Filter to patients who never had 'ED' in their list of visits
patients_without_ed = event_ids_per_patient[event_ids_per_patient.apply(lambda x: 'ED' not in x)]

# Step 3: Keep only those who had at least one non-ED
patients_with_only_non_ed = patients_without_ed[patients_without_ed.apply(lambda x: len(x) > 0)]

# Step 4: Show how many there are
print(f"✅ Number of patients with only non-'ED' EVENT_IDs: {len(patients_with_only_non_ed)}")

# Step 5 (optional): View a sample
df[df['PATNO'].isin(patients_with_only_non_ed.index)].sort_values(['PATNO', 'EVENT_ID'])


✅ Number of patients with only non-'ED' EVENT_IDs: 0


,REC_ID,PATNO,EVENT_ID,PAG_NAME,PDSURGDT,PDSURGTP,PDSRGTPC,PDSURGSD,PDSRGLOC,ORIG_ENTRY,LAST_UPDATE


Let's check one of these to confirm

In [25]:
# This patient doesn't have data stored as "ED"
df[df['PATNO'] == 3150]

,REC_ID,PATNO,EVENT_ID,PAG_NAME,PDSURGDT,PDSURGTP,PDSRGTPC,PDSURGSD,PDSRGLOC,ORIG_ENTRY,LAST_UPDATE
14,532077701,3150,V11,SURGPD,03/2015,1.000,NaN,1.000,5.000,08/2015,2020-06-30 09:43:52.0
197,532077701,3150,ED,SURGPD,03/2015,1.000,None,1.000,5.000,08/2015,2020-06-30 09:43:52.0


In [26]:
# Step 1: Group EVENT_IDs per patient
event_ids_per_patient = df.groupby("PATNO")["EVENT_ID"].unique()

# Step 2: Find patients with no "ED" and at least one other visit
patients_missing_ed = event_ids_per_patient[event_ids_per_patient.apply(
    lambda x: "ED" not in x and any(pd.notna(x))
)].index.tolist()

# Step 3: Select one row per such patient to duplicate
rows_to_duplicate = df[df["PATNO"].isin(patients_missing_ed)].groupby("PATNO").first().reset_index()

# Step 4: Create synthetic "ED" rows
synthetic_ed_rows = rows_to_duplicate.copy()
synthetic_ed_rows["EVENT_ID"] = "ED"

# Step 5: Merge with original dataset
tempdf = pd.concat([df, synthetic_ed_rows], ignore_index=True)
df = tempdf.copy()

# Step 6: Confirm results
print(f"✅ Added {len(synthetic_ed_rows)} synthetic 'ED' rows.")

# Step 7: Check if any PATNO has more than one ED visit
ed_counts = df[df["EVENT_ID"] == "ED"].groupby("PATNO").size()
patients_with_multiple_ed = ed_counts[ed_counts > 1]

if not patients_with_multiple_ed.empty:
    print(f"⚠️ {len(patients_with_multiple_ed)} patients have more than one 'ED' visit.")
else:
    print("✅ No patient has more than one 'ED' visit.")

print(df.shape)

✅ Added 0 synthetic 'ED' rows.
⚠️ 17 patients have more than one 'ED' visit.
(210, 11)


It works - now this and other patients have an "ED" timepoint

In [27]:
df[df['PATNO'] == 3150]

,REC_ID,PATNO,EVENT_ID,PAG_NAME,PDSURGDT,PDSURGTP,PDSRGTPC,PDSURGSD,PDSRGLOC,ORIG_ENTRY,LAST_UPDATE
14,532077701,3150,V11,SURGPD,03/2015,1.000,NaN,1.000,5.000,08/2015,2020-06-30 09:43:52.0
197,532077701,3150,ED,SURGPD,03/2015,1.000,None,1.000,5.000,08/2015,2020-06-30 09:43:52.0


Let's take a look at duplicates - as you can see, most of them are battery replacements, so picing the earliest is the best

In [28]:
# Get list of PATNOs with more than one ED
dup_patnos = patients_with_multiple_ed.index.tolist()

# Subset with only ED rows for those PATNOs
df_ed_duplicates = (
    df[(df["EVENT_ID"] == "ED") & (df["PATNO"].isin(dup_patnos))]
    .sort_values(["PATNO"])
    .copy()
)

print(f"Patients with more than one ED visit: {len(dup_patnos)}")
print("Shape of df_ed_duplicates:", df_ed_duplicates.shape)

df_ed_duplicates.head(10)

Patients with more than one ED visit: 17
Shape of df_ed_duplicates: (37, 11)


,REC_ID,PATNO,EVENT_ID,PAG_NAME,PDSURGDT,PDSURGTP,PDSRGTPC,PDSURGSD,PDSRGLOC,ORIG_ENTRY,LAST_UPDATE
6,IA98084,3078,ED,SURGPD,12/2021,3.000,DBS battery replacement,1.000,2.000,01/2022,2022-11-07 00:00:00.0
7,IA98083,3078,ED,SURGPD,06/2017,1.000,NaN,1.000,2.000,01/2022,2022-11-07 00:00:00.0
15,IA98125,3174,ED,SURGPD,01/2016,1.000,NaN,1.000,2.000,02/2022,2022-11-07 00:00:00.0
16,IA98124,3174,ED,SURGPD,04/2021,1.000,NaN,1.000,2.000,02/2022,2022-11-07 00:00:00.0
21,IA300864,3207,ED,SURGPD,01/2016,1.000,NaN,3.000,2.000,07/2023,2023-07-20 00:00:00.0
22,IA300863,3207,ED,SURGPD,01/2013,1.000,NaN,2.000,2.000,07/2023,2023-07-20 00:00:00.0
37,IA98100,3420,ED,SURGPD,10/2017,1.000,NaN,2.000,5.000,09/2020,2022-11-07 00:00:00.0
38,IA98099,3420,ED,SURGPD,10/2018,1.000,NaN,3.000,5.000,09/2020,2022-11-07 00:00:00.0
56,IA98094,3575,ED,SURGPD,01/2017,1.000,NaN,2.000,2.000,10/2021,2022-11-07 00:00:00.0
55,IA98093,3575,ED,SURGPD,01/2017,1.000,NaN,3.000,2.000,10/2021,2022-11-07 00:00:00.0


**Labels**

0 = no surgery (custom created)

1 = PDSURGTP (DBS)

2 - Levodopa intestinal gel

3 - Other

4 - Unknown

5 - HIFU

We need only 0 or 1 for the purposes of this script

Identifying and taking out patients that have duplicate ED (most likely DBS battery replacement)

In [29]:
# 1. Work on the full df and extract only ED rows
df_ed = df[df["EVENT_ID"] == "ED"].copy()

# 2. Restrict to ED rows with PDSURGTP == 1
df_ed_1 = df_ed[df_ed["PDSURGTP"] == 1].copy()

# 3. Parse PDSURGDT as month year (for example "06/2017")
df_ed_1["PDSURGDT_parsed"] = pd.to_datetime(
    df_ed_1["PDSURGDT"],
    format="%m/%Y",
    errors="coerce"
)

# 4. Sort by PATNO and surgery date (oldest first)
df_ed_1 = df_ed_1.sort_values(["PATNO", "PDSURGDT_parsed"])

# 5. Keep only the earliest ED per PATNO among PDSURGTP == 1
dfsubset_earliest = (
    df_ed_1
    .drop_duplicates(subset="PATNO", keep="first")
)

# 6. Rows that were not kept (either PDSURGTP != 1, or later ED rows with PDSURGTP == 1)
kept_index = dfsubset_earliest.index
df_removed = df_ed.loc[~df_ed.index.isin(kept_index)].copy()

# 7. Drop helper column from dfsubset_earliest
dfsubset_earliest = dfsubset_earliest.drop(columns=["PDSURGDT_parsed"])

# 8. Create dfsubset from earliest rows with PDSURGTP == 1
dfsubset = dfsubset_earliest.copy()

print("Original ED rows (from df):", df_ed.shape)
print("ED rows with PDSURGTP == 1:", df_ed_1.shape)
print("After keeping earliest ED per PATNO with PDSURGTP == 1 (dfsubset):", dfsubset.shape)
print("Rows removed (df_removed):", df_removed.shape)

dfsubset.head()


Original ED rows (from df): (147, 11)
ED rows with PDSURGTP == 1: (136, 12)
After keeping earliest ED per PATNO with PDSURGTP == 1 (dfsubset): (121, 11)
Rows removed (df_removed): (26, 11)


,REC_ID,PATNO,EVENT_ID,PAG_NAME,PDSURGDT,PDSURGTP,PDSRGTPC,PDSURGSD,PDSRGLOC,ORIG_ENTRY,LAST_UPDATE
0,IA98081,3010,ED,SURGPD,08/2016,1.000,NaN,1.000,5.000,12/2021,2022-11-07 00:00:00.0
3,IA445027,3061,ED,SURGPD,12/2015,1.000,NaN,1.000,5.000,03/2024,2024-03-14 00:00:00.0
195,550512501,3066,ED,SURGPD,11/2015,1.000,None,1.000,1.000,01/2016,2020-06-22 09:28:47.0
7,IA98083,3078,ED,SURGPD,06/2017,1.000,NaN,1.000,2.000,01/2022,2022-11-07 00:00:00.0
10,IA98086,3122,ED,SURGPD,06/2020,1.000,NaN,1.000,2.000,04/2022,2022-11-07 00:00:00.0


## Adding patients who haven't done surgery to the dataset

As per our previous transformations, only patients that underwent surgery are added. However, having a full dataset with non-operated cases is relevant for comparisons analyses

In [30]:
COHORT_FILE = get_latest_file(prefix="Participant_Status", directory=DATA_DIR)
cohort = pd.read_csv(COHORT_FILE)
print('Length of the dataset:', len(cohort))
cohort.head()

Looking for files in: G:\Meu Drive\Colab\Useful-PPMI-Clinical-Codes\data
Found files: ['G:\\Meu Drive\\Colab\\Useful-PPMI-Clinical-Codes\\data\\Participant_Status_01Aug2025.csv']
Latest file: G:\Meu Drive\Colab\Useful-PPMI-Clinical-Codes\data\Participant_Status_01Aug2025.csv
Length of the dataset: 7401


,PATNO,COHORT,COHORT_DEFINITION,ENROLL_DATE,ENROLL_STATUS,STATUS_DATE,SCREENEDAM,ENROLL_AGE,INEXPAGE,AV133STDY,TAUSTDY,GAITSTDY,PISTDY,SV2ASTDY,NXTAUSTDY,DATELIG,PPMI_ONLINE_ENROLL,ENRLPINK1,ENRLPRKN,ENRLSRDC,ENRLNORM,ENRLOTHGV,ENRLHPSM,ENRLRBD,ENRLLRRK2,ENRLSNCA,ENRLGBA
0,3000,2,Healthy Control,02/2011,Withdrew,10/2024,NaN,69.100,NaN,0.000,0.000,0.000,0.000,0.000,0.000,NaN,NO,0.000,0.000,0.000,NaN,NaN,0,0,0,0,0
1,3001,1,Parkinson's Disease,03/2011,Enrolled,09/2021,NaN,65.100,NaN,0.000,0.000,0.000,0.000,0.000,0.000,NaN,NO,0.000,0.000,1.000,NaN,NaN,0,0,0,0,0
2,3002,1,Parkinson's Disease,03/2011,Withdrew,10/2024,NaN,67.600,NaN,0.000,0.000,0.000,0.000,0.000,0.000,NaN,NO,0.000,0.000,1.000,NaN,NaN,0,0,0,0,0
3,3003,1,Parkinson's Disease,04/2011,Enrolled,01/2022,NaN,56.700,NaN,0.000,0.000,0.000,0.000,0.000,0.000,NaN,YES,0.000,0.000,1.000,NaN,NaN,0,0,0,0,0
4,3004,2,Healthy Control,04/2011,Enrolled,01/2022,NaN,59.400,NaN,0.000,0.000,1.000,0.000,0.000,0.000,NaN,YES,0.000,0.000,0.000,NaN,NaN,0,0,0,0,0


In [31]:
# 1. Subset cohort to PD only
cohort_pd = cohort[cohort["COHORT_DEFINITION"] == "Parkinson's Disease"].copy()

# 2. PATNO sets
pd_patnos = set(cohort_pd["PATNO"])
existing_patnos = set(dfsubset["PATNO"])

# 3. PD patients missing from dfsubset
missing_patnos = sorted(pd_patnos - existing_patnos)

print(f"Number of PD patients in cohort: {len(pd_patnos)}")
print(f"Already in dfsubset: {len(existing_patnos)}")
print(f"PD patients missing in dfsubset: {len(missing_patnos)}")

# 4. Mark existing rows as already present
dfsubset_extended = dfsubset.copy()
dfsubset_extended["ED_status"] = "Already_present"

# 5. Build rows for missing PATNOs
new_rows = pd.DataFrame({
    "REC_ID": np.nan,
    "PATNO": missing_patnos,
    "EVENT_ID": "ED",
    "PAG_NAME": "SURGPD",
    "PDSURGDT": np.nan,
    "PDSURGTP": 0,
    "PDSRGTPC": np.nan,
    "PDSURGSD": np.nan,
    "PDSRGLOC": np.nan,
    "ORIG_ENTRY": np.nan,
    "LAST_UPDATE": np.nan,
    "ED_status": "Added"
})

# 6. Combine existing and new rows
dfsubset = pd.concat([dfsubset_extended, new_rows], ignore_index=True)

# Optional: sort by PATNO
dfsubset = dfsubset.sort_values(["PATNO", "EVENT_ID"]).reset_index(drop=True)

print("Final dfsubset shape:", dfsubset.shape)
print(dfsubset["ED_status"].value_counts())
dfsubset.head()


Number of PD patients in cohort: 1919
Already in dfsubset: 121
PD patients missing in dfsubset: 1799
Final dfsubset shape: (1920, 12)
ED_status
Added              1799
Already_present     121
Name: count, dtype: int64


,REC_ID,PATNO,EVENT_ID,PAG_NAME,PDSURGDT,PDSURGTP,PDSRGTPC,PDSURGSD,PDSRGLOC,ORIG_ENTRY,LAST_UPDATE,ED_status
0,NaN,3001,ED,SURGPD,NaN,0.000,NaN,NaN,NaN,NaN,NaN,Added
1,NaN,3002,ED,SURGPD,NaN,0.000,NaN,NaN,NaN,NaN,NaN,Added
2,NaN,3003,ED,SURGPD,NaN,0.000,NaN,NaN,NaN,NaN,NaN,Added
3,NaN,3005,ED,SURGPD,NaN,0.000,NaN,NaN,NaN,NaN,NaN,Added
4,NaN,3006,ED,SURGPD,NaN,0.000,NaN,NaN,NaN,NaN,NaN,Added


## Standardizing timepoints

Loading MDS III dataset (will be used to align longitudinal timepoints to PPMI's EVENT_ID)

In [32]:
PPMI_FILE = get_latest_file(prefix="MDS-UPDRS_Part_III", directory=DATA_DIR)
ppmiupdrs = pd.read_csv(PPMI_FILE)
print('Length of the dataset:', len(ppmiupdrs))
ppmiupdrs.head()

Looking for files in: G:\Meu Drive\Colab\Useful-PPMI-Clinical-Codes\data
Found files: ['G:\\Meu Drive\\Colab\\Useful-PPMI-Clinical-Codes\\data\\MDS-UPDRS_Part_III_09Feb2025.csv']
Latest file: G:\Meu Drive\Colab\Useful-PPMI-Clinical-Codes\data\MDS-UPDRS_Part_III_09Feb2025.csv
Length of the dataset: 31767


,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,PDTRTMNT,PDSTATE,HRPOSTMED,HRDBSON,HRDBSOFF,PDMEDYN,DBSYN,ONOFFORDER,OFFEXAM,OFFNORSN,DBSOFFTM,ONEXAM,ONNORSN,HIFUYN,DBSONTM,PDMEDDT,PDMEDTM,EXAMDT,EXAMTM,NP3SPCH,NP3FACXP,NP3RIGN,NP3RIGRU,NP3RIGLU,NP3RIGRL,NP3RIGLL,NP3FTAPR,NP3FTAPL,NP3HMOVR,NP3HMOVL,NP3PRSPR,NP3PRSPL,NP3TTAPR,NP3TTAPL,NP3LGAGR,NP3LGAGL,NP3RISNG,NP3GAIT,NP3FRZGT,NP3PSTBL,NP3POSTR,NP3BRADY,NP3PTRMR,NP3PTRML,NP3KTRMR,NP3KTRML,NP3RTARU,NP3RTALU,NP3RTARL,NP3RTALL,NP3RTALJ,NP3RTCON,NP3TOT,DYSKPRES,DYSKIRAT,NHY,ORIG_ENTRY,LAST_UPDATE
0,272451901,3000,BL,NUPDRS3,02/2011,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02/2011,13:17:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,NaN,0.000,02/2011,2020-06-25 16:02:19.0
1,338703101,3000,V04,NUPDRS3,03/2012,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/2012,13:47:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,NaN,0.000,03/2012,2020-06-25 16:02:22.0
2,385009801,3000,V06,NUPDRS3,02/2013,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02/2013,12:22:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,NaN,0.000,02/2013,2020-06-25 16:02:22.0
3,437131401,3000,V08,NUPDRS3,03/2014,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/2014,13:22:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.000,0.000,NaN,0.000,05/2014,2020-06-25 16:02:22.0
4,512469901,3000,V10,NUPDRS3,03/2015,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/2015,11:43:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,2.000,2.000,1.000,1.000,1.000,2.000,1.000,2.000,0.000,0.000,0.000,0.000,1.000,0.000,1.000,1.000,1.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,19.000,0.000,NaN,0.000,03/2015,2020-06-25 16:02:23.0


First, let's drop duplicate rows for the same timepoint per PATNO (most of them are ON/OFF testing, but we don't care about any values here. We are just using this dataset as a proxy to have the timepoints for each patient)

In [33]:
# Number of rows before dropping duplicates
before = ppmiupdrs.shape[0]

# Drop duplicates
ppmiupdrs = ppmiupdrs.drop_duplicates(subset=["PATNO", "EVENT_ID"], keep="first")

# Number of rows after
after = ppmiupdrs.shape[0]

# How many were removed
dropped = before - after
print(f"Rows before: {before}")
print(f"Rows after:  {after}")
print(f"Rows dropped: {dropped}")


Rows before: 31767
Rows after:  26400
Rows dropped: 5367


Let's just subset what we need

In [34]:
ppmiupdrs = ppmiupdrs[['PATNO','EVENT_ID','INFODT']]
ppmiupdrs.head()

,PATNO,EVENT_ID,INFODT
0,3000,BL,02/2011
1,3000,V04,03/2012
2,3000,V06,02/2013
3,3000,V08,03/2014
4,3000,V10,03/2015


For this code to work, we will subset the dataset to standard follow-ups

In [35]:
event_id_mapping = {
    'BL': 0,
    'V01': 0.25,
    'V02': 0.5,
    'V03': 0.75,
    'V04': 1,
    'V05': 1.5,
    'V06': 2,
    'V07': 2.5,
    'V08': 3,
    'V09': 3.5,
    'V10': 4,
    'V11': 4.5,
    'V12': 5,
    'V13': 6,
    'V14': 7,
    'V15': 8,
    'V16': 9,
    'V17': 10,
    'V18': 11,
    'V19': 12,
    'V20': 13,
    'V21': 14,
    'V22': 15,
    'V23': 16
}

# Keep only rows whose EVENT_ID is one of BL, V01, V02, ...
valid_events = set(event_id_mapping.keys())

ppmiupdrs = ppmiupdrs[ppmiupdrs["EVENT_ID"].isin(valid_events)].copy()

print("Unique EVENT_ID after filtering:", sorted(ppmiupdrs["EVENT_ID"].unique()))
ppmiupdrs[["PATNO", "EVENT_ID"]].head()


Unique EVENT_ID after filtering: ['BL', 'V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21']


,PATNO,EVENT_ID
0,3000,BL
1,3000,V04
2,3000,V06
3,3000,V08
4,3000,V10


In [36]:
# Let's rename some cols to work better
ppmiupdrs = ppmiupdrs.rename(columns={
    "EVENT_ID": "EVENT_ID_MDS",
    "INFODT": "INFODT_MDS"
})

ppmiupdrs.head()


,PATNO,EVENT_ID_MDS,INFODT_MDS
0,3000,BL,02/2011
1,3000,V04,03/2012
2,3000,V06,02/2013
3,3000,V08,03/2014
4,3000,V10,03/2015


In [37]:
# Merging: add UPDRS data (ppmiupdrs) to dfsubset
df = pd.merge(dfsubset, ppmiupdrs, on="PATNO", how="inner")

# Unique PATNO counts
n_left   = dfsubset["PATNO"].nunique()
n_right  = ppmiupdrs["PATNO"].nunique()
n_merged = df["PATNO"].nunique()

# Shapes and PATNO counts
print(f"Left dataset (dfsubset):   shape = {dfsubset.shape},  unique PATNO = {n_left}")
print(f"Right dataset (ppmiupdrs): shape = {ppmiupdrs.shape}, unique PATNO = {n_right}")
print(f"Merged dataset (df):       shape = {df.shape},        unique PATNO = {n_merged}")

# Proportion of left dataset PATNO retained in merged
retained_pct = (n_merged / n_left) * 100
print(f"PATNO retained from left into merged: {n_merged}/{n_left} ({retained_pct:.2f}%)")

df.head()


Left dataset (dfsubset):   shape = (1920, 12),  unique PATNO = 1920
Right dataset (ppmiupdrs): shape = (21059, 3), unique PATNO = 4067
Merged dataset (df):       shape = (11192, 14),        unique PATNO = 1473
PATNO retained from left into merged: 1473/1920 (76.72%)


,REC_ID,PATNO,EVENT_ID,PAG_NAME,PDSURGDT,PDSURGTP,PDSRGTPC,PDSURGSD,PDSRGLOC,ORIG_ENTRY,LAST_UPDATE,ED_status,EVENT_ID_MDS,INFODT_MDS
0,NaN,3001,ED,SURGPD,NaN,0.000,NaN,NaN,NaN,NaN,NaN,Added,BL,03/2011
1,NaN,3001,ED,SURGPD,NaN,0.000,NaN,NaN,NaN,NaN,NaN,Added,V01,05/2011
2,NaN,3001,ED,SURGPD,NaN,0.000,NaN,NaN,NaN,NaN,NaN,Added,V02,08/2011
3,NaN,3001,ED,SURGPD,NaN,0.000,NaN,NaN,NaN,NaN,NaN,Added,V03,11/2011
4,NaN,3001,ED,SURGPD,NaN,0.000,NaN,NaN,NaN,NaN,NaN,Added,V04,03/2012


This new dataset now has several rows for each patient with the timepoints from the MDS and the corresponding time in years to that timepoint. We will clean this later

## Time since disease onset

Now will add to the dataset information on how many years it took for the patient to do the surgery and how many years it took since onset and all follow-ups

In [38]:
DIAGNOSIS_FILE = get_latest_file(prefix="PD_Diagnosis_History", directory=DATA_DIR)
dxtime = pd.read_csv(DIAGNOSIS_FILE)
print('Length of the dataset:', len(dxtime))
dxtime.head()

Looking for files in: G:\Meu Drive\Colab\Useful-PPMI-Clinical-Codes\data
Found files: ['G:\\Meu Drive\\Colab\\Useful-PPMI-Clinical-Codes\\data\\PD_Diagnosis_History_07Oct2024.csv']
Latest file: G:\Meu Drive\Colab\Useful-PPMI-Clinical-Codes\data\PD_Diagnosis_History_07Oct2024.csv
Length of the dataset: 1910


,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,SXDT,PDDXDT,DXTREMOR,DXRIGID,DXBRADY,DXPOSINS,DXOTHSX,DOMSIDE,ORIG_ENTRY,LAST_UPDATE
0,274786201,3001,SC,PDFEAT,02/2011,08/2009,04/2010,1.000,1.000,1.000,0.000,0.000,2.000,02/2011,2020-06-25 16:04:32.0
1,278719301,3002,SC,PDFEAT,03/2011,02/2009,02/2010,1.000,0.000,1.000,0.000,1.000,1.000,03/2011,2020-04-01 17:33:44.0
2,281142401,3003,SC,PDFEAT,03/2011,05/2006,03/2009,0.000,1.000,1.000,1.000,1.000,2.000,03/2011,2020-06-25 16:06:25.0
3,281508301,3005,SC,PDFEAT,03/2011,08/2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/2011,2020-02-11 15:00:24.0
4,283723801,3006,SC,PDFEAT,03/2011,12/2009,11/2010,1.000,1.000,1.000,0.000,0.000,2.000,03/2011,2020-04-22 09:14:40.0


In [39]:
# Best to use PDDXDT (more data) - PD Diagnosis
# SXDT = Symptom onset
dxtime[['SXDT','PDDXDT']].describe(include='all')

,SXDT,PDDXDT
count,1888,1906
unique,230,190
top,01/2021,01/2023
freq,51,37


Check for missingness in INFO_DT_MDS (for later calcs)

In [40]:
print(df["INFODT_MDS"].isna().sum())
df[df["INFODT_MDS"].isna()].head(10)

0


,REC_ID,PATNO,EVENT_ID,PAG_NAME,PDSURGDT,PDSURGTP,PDSRGTPC,PDSURGSD,PDSRGLOC,ORIG_ENTRY,LAST_UPDATE,ED_status,EVENT_ID_MDS,INFODT_MDS


Mergining datasets

In [41]:
# Merging: add PDDXDT from dxtime to df
df_final = pd.merge(
    df,
    dxtime[["PATNO", "PDDXDT"]],
    on="PATNO",
    how="inner"
)

# Unique PATNO counts
n_left   = df["PATNO"].nunique()
n_right  = dxtime["PATNO"].nunique()
n_merged = df_final["PATNO"].nunique()

# Shapes
print(f"Left dataset (df):           shape = {df.shape},    unique PATNO = {n_left}")
print(f"Right dataset (dxtime):      shape = {dxtime.shape}, unique PATNO = {n_right}")
print(f"Merged dataset (df_final):   shape = {df_final.shape}, unique PATNO = {n_merged}")

# Proportion of left dataset PATNO retained in merged
retained_pct = (n_merged / n_left) * 100
print(f"PATNO retained from left dataset into merged: {n_merged}/{n_left} ({retained_pct:.2f}%)")

df_final.head()


Left dataset (df):           shape = (11192, 14),    unique PATNO = 1473
Right dataset (dxtime):      shape = (1910, 15), unique PATNO = 1910
Merged dataset (df_final):   shape = (11185, 15), unique PATNO = 1466
PATNO retained from left dataset into merged: 1466/1473 (99.52%)


,REC_ID,PATNO,EVENT_ID,PAG_NAME,PDSURGDT,PDSURGTP,PDSRGTPC,PDSURGSD,PDSRGLOC,ORIG_ENTRY,LAST_UPDATE,ED_status,EVENT_ID_MDS,INFODT_MDS,PDDXDT
0,NaN,3001,ED,SURGPD,NaN,0.000,NaN,NaN,NaN,NaN,NaN,Added,BL,03/2011,04/2010
1,NaN,3001,ED,SURGPD,NaN,0.000,NaN,NaN,NaN,NaN,NaN,Added,V01,05/2011,04/2010
2,NaN,3001,ED,SURGPD,NaN,0.000,NaN,NaN,NaN,NaN,NaN,Added,V02,08/2011,04/2010
3,NaN,3001,ED,SURGPD,NaN,0.000,NaN,NaN,NaN,NaN,NaN,Added,V03,11/2011,04/2010
4,NaN,3001,ED,SURGPD,NaN,0.000,NaN,NaN,NaN,NaN,NaN,Added,V04,03/2012,04/2010


One relevant note: we do lose data on some patients that didn't do surgery by requiring information on diagnosis date. This can be adapted by not requiring this. However, I think this is relevant and we don't lose information on patients that did undergo surgery

In [42]:
# Confirming we still keep the same number of patients that were operated - see below
df_final.groupby("ED_status")["PATNO"].nunique()

ED_status
Added              1345
Already_present     121
Name: PATNO, dtype: int64

Now we create two useful columns that will tell us how many years have passed since that patient has done surgery or between his follow-up

In [43]:
# Convert date columns to datetime format
df_final['PDDXDT'] = pd.to_datetime(df_final['PDDXDT'], format='%m/%Y', errors='coerce')
df_final['INFODT_MDS'] = pd.to_datetime(df_final['INFODT_MDS'], format='%m/%Y', errors='coerce')
df_final['PDSURGDT'] = pd.to_datetime(df_final['PDSURGDT'], format='%m/%Y', errors='coerce')

# Calculate the difference in years, keeping NaNs where PDSURGDT is missing
df_final['Years_between_diagnosis_and_surgery'] = (df_final['PDSURGDT'] - df_final['PDDXDT']).dt.days / 365.25
df_final['Years_between_diagnosis_follow_up'] = (df_final['INFODT_MDS'] - df_final['PDDXDT']).dt.days / 365.25

# Display the updated dataset
df_final.head()

,REC_ID,PATNO,EVENT_ID,PAG_NAME,PDSURGDT,PDSURGTP,PDSRGTPC,PDSURGSD,PDSRGLOC,ORIG_ENTRY,LAST_UPDATE,ED_status,EVENT_ID_MDS,INFODT_MDS,PDDXDT,Years_between_diagnosis_and_surgery,Years_between_diagnosis_follow_up
0,NaN,3001,ED,SURGPD,NaT,0.000,NaN,NaN,NaN,NaN,NaN,Added,BL,2011-03-01,2010-04-01,NaN,0.914
1,NaN,3001,ED,SURGPD,NaT,0.000,NaN,NaN,NaN,NaN,NaN,Added,V01,2011-05-01,2010-04-01,NaN,1.081
2,NaN,3001,ED,SURGPD,NaT,0.000,NaN,NaN,NaN,NaN,NaN,Added,V02,2011-08-01,2010-04-01,NaN,1.333
3,NaN,3001,ED,SURGPD,NaT,0.000,NaN,NaN,NaN,NaN,NaN,Added,V03,2011-11-01,2010-04-01,NaN,1.585
4,NaN,3001,ED,SURGPD,NaT,0.000,NaN,NaN,NaN,NaN,NaN,Added,V04,2012-03-01,2010-04-01,NaN,1.916


In [44]:
df_final = df_final[['PATNO','PDSURGTP','ED_status','EVENT_ID_MDS','EVENT_ID','INFODT_MDS','Years_between_diagnosis_and_surgery','Years_between_diagnosis_follow_up']]

In [45]:
df_final.head()

,PATNO,PDSURGTP,ED_status,EVENT_ID_MDS,EVENT_ID,INFODT_MDS,Years_between_diagnosis_and_surgery,Years_between_diagnosis_follow_up
0,3001,0.000,Added,BL,ED,2011-03-01,NaN,0.914
1,3001,0.000,Added,V01,ED,2011-05-01,NaN,1.081
2,3001,0.000,Added,V02,ED,2011-08-01,NaN,1.333
3,3001,0.000,Added,V03,ED,2011-11-01,NaN,1.585
4,3001,0.000,Added,V04,ED,2012-03-01,NaN,1.916


## Correct data cleaning and exporting

This new and ultra-merged dataset has several rows for each patient regarding their surgery due to the MDS merge. This is intentional as we will use this to calculate timepoints in which the patient didn't had surgery yet and those that he did.

The goal of this part of the code is to provide: (1) a longitudinal dataset informing when specific individuals went DBS surgery and (2) a cross-sectional dataset informing who underwent surgery and who didn't

The first thing we will do is to make it so that "PDSURGTP" is put as 0 for all timepoints that happened before surgery, and 1 after surgery happened. This is relevant so that longitudinal survival analysis models can use the dataset (such as Cox Proportional Hazards)

In [46]:
dbs_longitudinal = df_final.copy()

# Apply condition: Set PDSURGTP to 0 if Years_between_onset_follow_up < Years_between_onset_and_surgery
dbs_longitudinal["PDSURGTP"] = df_final.apply(
    lambda row: 0 if row["Years_between_diagnosis_follow_up"] < row["Years_between_diagnosis_and_surgery"] else row["PDSURGTP"],
    axis=1
)

# Find the first PATNO with ED_status == "Already_present"
# This is just to showcase a patient with DBS surgery without mentioning his PATNO
first_patno_already = dbs_longitudinal.loc[dbs_longitudinal["ED_status"] == "Already_present", "PATNO"].iloc[0]

# Display all columns for that patient
dbs_longitudinal[dbs_longitudinal["PATNO"] == first_patno_already].head(50)

,PATNO,PDSURGTP,ED_status,EVENT_ID_MDS,EVENT_ID,INFODT_MDS,Years_between_diagnosis_and_surgery,Years_between_diagnosis_follow_up
58,3010,0.000,Already_present,BL,ED,2011-06-01,5.498,0.329
59,3010,0.000,Already_present,V01,ED,2011-09-01,5.498,0.580
60,3010,0.000,Already_present,V02,ED,2012-01-01,5.498,0.914
61,3010,0.000,Already_present,V03,ED,2012-03-01,5.498,1.079
62,3010,0.000,Already_present,V04,ED,2012-06-01,5.498,1.331
63,3010,0.000,Already_present,V05,ED,2013-01-01,5.498,1.916
64,3010,0.000,Already_present,V06,ED,2013-07-01,5.498,2.412
65,3010,0.000,Already_present,V07,ED,2014-01-01,5.498,2.916
66,3010,0.000,Already_present,V08,ED,2014-09-01,5.498,3.581
67,3010,0.000,Already_present,V09,ED,2014-12-01,5.498,3.830


As you can see, for this patient that underwent surgery, there is a shift in PDSURGTP from 0 to 1 at a specific timepoint. And the longitudinal dataset is now ready

Now the cross sectional dataset. We will subset the ED of the latest follow-up. That way, we also have information on how many years have passed since the patient's diagnosis so that we can make better comparisons between DBS surgery or not (a clear confounder for these analysis is this variable, since, usually, patients that didn't follow-up at least 3-4 years didn't had a chance to undergo surgery, as this is only indicated after this amount of time)

In [47]:
# Remove rows where INFODT_MDS is NaN
df_cross = dbs_longitudinal[dbs_longitudinal["INFODT_MDS"].notna()].copy()

# Keep only EVENT_ID == "ED"
df_cross = df_cross[df_cross["EVENT_ID"] == "ED"].copy()
print("After INFODT_MDS notna and EVENT_ID == 'ED':", df_cross.shape)

# Drop duplicates, keeping the last by PATNO
df_cross = df_cross.drop_duplicates(subset=["PATNO"], keep="last")

print("Final df_cross shape:", df_cross.shape)
df_cross.head()


After INFODT_MDS notna and EVENT_ID == 'ED': (11185, 8)
Final df_cross shape: (1466, 8)


,PATNO,PDSURGTP,ED_status,EVENT_ID_MDS,EVENT_ID,INFODT_MDS,Years_between_diagnosis_and_surgery,Years_between_diagnosis_follow_up
18,3001,0.000,Added,V20,ED,2024-09-01,NaN,14.420
34,3002,0.000,Added,V18,ED,2022-03-01,NaN,12.077
51,3003,0.000,Added,V20,ED,2024-05-01,NaN,15.168
56,3006,0.000,Added,V04,ED,2012-05-01,NaN,1.498
57,3007,0.000,Added,BL,ED,2011-05-01,NaN,0.329


Now to export both of the datasets

In [48]:
# Exporting
dbs_longitudinal.to_csv(os.path.join(PRIV_DIR, "DBS_surgery_cross_longitudinal.csv"), index=False)
df_cross.to_csv(os.path.join(PRIV_DIR, "DBS_surgery_cross_sectional.csv"), index=False)